# Task4

In [1]:
from qiskit import QuantumRegister, ClassicalRegister
from qiskit import QuantumCircuit, execute, Aer
import pennylane as qml
import numpy as np
from numpy import pi
simulator = Aer.get_backend('qasm_simulator')



# Matrix and Decomposition

In [59]:
M= np.matrix('0 -1  ;-1 0')

In [60]:
coeffs, obs_list = qml.utils.decompose_hamiltonian(M)
H = qml.Hamiltonian(coeffs, obs_list)

In [61]:
print(H)

(-1.0) [X0]


# Ansatz

In [9]:
def Operational_circuit(params):
    qc = QuantumCircuit(1)
    
    #qc.h(range(2))
    qc.ry(params[0],0)
    
    return qc
    

In [21]:
Operational_circuit_try = Operational_circuit([0])

In [22]:
Operational_circuit_try.draw()

┌───────┐
q_0: ┤ RY(0) ├
     └───────┘

# Measurments for different components of H

# 1. Z

In [23]:
def measure_z_circuit(given_circuit):
    z_meas = given_circuit.copy()
    z_meas.measure_all()
    return z_meas

z_meas = measure_z_circuit(Operational_circuit_try)
z_meas.draw()

┌───────┐ ░ ┌─┐
   q_0: ┤ RY(0) ├─░─┤M├
        └───────┘ ░ └╥┘
meas_0: ═════════════╩═

In [24]:
def measure_z(given_circuit, num_shots = 10000):

    z_meas = measure_z_circuit(given_circuit)
    
    result = execute(z_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(z_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1']
    z = counts['0'] - counts['1']
    z = z / total_counts
    
    return z

In [25]:
z = measure_z(Operational_circuit_try)
print("<Z> =", str(z))

<Z> = 1.0


# 2. X

In [28]:
def measure_x_circuit(given_circuit):
    x_meas = given_circuit.copy()
    x_meas.h(0)
    x_meas.measure_all()

    return x_meas

In [29]:
x_meas = measure_x_circuit(Operational_circuit_try)
x_meas.draw()

┌───────┐┌───┐ ░ ┌─┐
   q_0: ┤ RY(0) ├┤ H ├─░─┤M├
        └───────┘└───┘ ░ └╥┘
meas_0: ══════════════════╩═

In [32]:
def measure_x(given_circuit, num_shots = 10000):
    
    x_meas = measure_x_circuit(given_circuit)
    
    result = execute(x_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(x_meas)

    if '0' not in counts:
        counts['0'] = 0
    if '1' not in counts:
        counts['1'] = 0

    total_counts = counts['0'] + counts['1'] 
    x = counts['0'] - counts['1']
    x = x / total_counts
    
    return x

In [33]:
x = measure_x(Operational_circuit_try)
print("<X> =", str(x))

<X> = -0.0182


# 3. Y

In [34]:
def measure_y_circuit(given_circuit):
    y_meas = given_circuit.copy()
    y_meas.rz(-pi/2,0)
    y_meas.h(0)

    y_meas.measure_all()

    return y_meas

In [35]:
y_meas = measure_y_circuit(Operational_circuit_try)
y_meas.draw()

┌───────┐┌───────────┐┌───┐ ░ ┌─┐
   q_0: ┤ RY(0) ├┤ RZ(-pi/2) ├┤ H ├─░─┤M├
        └───────┘└───────────┘└───┘ ░ └╥┘
meas_0: ═══════════════════════════════╩═

In [36]:
def measure_y(given_circuit, num_shots = 10000):
    
    yy_meas = measure_y_circuit(given_circuit)
    
    result = execute(y_meas, backend = simulator, shots = num_shots).result()
    counts = result.get_counts(y_meas)

    if '0' not in counts:
        counts['00'] = 0
    if '1' not in counts:
        counts['01'] = 0

    total_counts = counts['0'] + counts['1'] 
    y = counts['0']  - counts['1']
    y = y / total_counts
    
    return y

In [37]:
y = measure_y(Operational_circuit_try)
print("<y> =", str(y))

<y> = 0.0148


# Cost_Function

In [63]:
print(H)

(-1.0) [X0]


In [62]:
def eigen_value(params):
    num_shots = 10000
    x = measure_x(Operational_circuit(params), num_shots = num_shots)
    ev = -x
    
    return ev

In [64]:
params = [0]

In [65]:
ev = eigen_value(params)     #Try energy with params = [0,0,0,0,0,0]
print("The Eigen_Value of the trial state is", str(ev))

The Eigen_Value of the trial state is 0.0104


# Finding Lowest Eignevalue

In [66]:
from qiskit.aqua.components.optimizers import COBYLA,SLSQP,SPSA

optimizer = COBYLA(50,0.001)

In [67]:
ret = optimizer.optimize(num_vars=1, objective_function=eigen_value, initial_point=params)

In [68]:
eigen_value(ret[0])

-1.0

# Eigen State.

In [52]:
backend_state = Aer.get_backend('statevector_simulator')

In [69]:

result = execute(Operational_circuit(ret[0]),backend_state).result()
out_state = result.get_statevector()


In [71]:
print(out_state)

[0.71003388+0.j 0.70416751+0.j]


In [72]:
qc = QuantumCircuit(1)
qc.initialize(out_state,0)

In [75]:
def eigen_value_state(qc):
    num_shots = 10000
    x = measure_x(qc, num_shots = num_shots)
    ev = -x
    
    return ev

In [76]:
 eigen_value_state(qc)

-1.0